In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import GRU, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
list_sentences_train[:5]

array([ 'explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now 89 205 38 27',
       'd aww ! he matches this background colour i am seemingly stuck with thanks talk 21 51 january 11 2016 utc ',
       'hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info ',
       ' more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting styl

In [3]:
print('test len',len(test))

test len 153164


In [4]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(159571, 150) (153164, 150)


In [5]:
# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

turbocharger 173243
range 1561
thole 151177
kxii 49438
miscegenated 129949
kherau 154650


In [6]:
# read word2vec
# 
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
# print(word_vec_dict['is'])
# print(word_vec_dict['are'])

print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

2196007
Preparing embedding matrix
Null word embeddings: 3388


In [7]:
from sklearn.metrics import log_loss,accuracy_score
from keras.layers import MaxPooling1D

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_lstm_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = Dropout(0.2)(x)
    main = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    main = MaxPooling1D(pool_size=2)(main)
    main = GRU(64)(main)
    main = Dense(32, activation="relu")(main)
    main = Dense(6, activation="sigmoid")(main)
    model = Model(inputs=inp, outputs=main)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
    return model
    

print('def model done')

def model done


In [8]:
tmp_m = get_lstm_model()
tmp_m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 300)          12000000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 300)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 150, 256)          230656    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 75, 256)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                61632     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
__________

In [10]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    # (159571, 150) (153164, 150)

    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_lstm_model()
        batch_size = 64
        epochs = 8
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/cnn_gru_glove_1_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/cnn_gru_glove_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')

# pre 0.04337 0.04435 0.04584
# add embed dropout
# new 0.04377 0.04312 0.04517

Train on 106380 samples, validate on 53191 samples
Epoch 1/8
106380/106380 [==============================] - 124s 1ms/step - loss: 0.0540 - acc: 0.9807 - val_loss: 0.0450 - val_acc: 0.9826
Epoch 2/8
106380/106380 [==============================] - 124s 1ms/step - loss: 0.0433 - acc: 0.9833 - val_loss: 0.0440 - val_acc: 0.9829
Epoch 3/8
106380/106380 [==============================] - 125s 1ms/step - loss: 0.0397 - acc: 0.9847 - val_loss: 0.0421 - val_acc: 0.9836
Epoch 4/8
106380/106380 [==============================] - 125s 1ms/step - loss: 0.0368 - acc: 0.9855 - val_loss: 0.0434 - val_acc: 0.9836
Epoch 5/8
106380/106380 [==============================] - 125s 1ms/step - loss: 0.0343 - acc: 0.9863 - val_loss: 0.0433 - val_acc: 0.9829
Epoch 6/8
106380/106380 [==============================] - 125s 1ms/step - loss: 0.0315 - acc: 0.9873 - val_loss: 0.0445 - val_acc: 0.9829
Epoch 7/8
106380/106380 [==============================] - 125s 1ms/step - loss: 0.0292 - acc: 0.9884 - val_loss: 0

In [11]:
train_pred[:10].round(3)

array([[ 0.015,  0.   ,  0.002,  0.   ,  0.002,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.006,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.023,  0.   ,  0.003,  0.001,  0.005,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.993,  0.273,  0.971,  0.007,  0.772,  0.014],
       [ 0.03 ,  0.   ,  0.001,  0.001,  0.001,  0.   ],
       [ 0.017,  0.   ,  0.   ,  0.   ,  0.001,  0.   ],
       [ 0.003,  0.   ,  0.   ,  0.   ,  0.001,  0.   ]])

In [12]:
y[:10]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])